In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install chart_studio

     |████████████████████████████████| 71kB 3.3MB/s 


In [ ]:
#importing the libraries
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
import os
import re
import numpy as np
import pandas as pd
import chart_studio.plotly as py
import plotly.graph_objs as go
import itertools
from scipy import stats
from ast import literal_eval
import spacy
from tensorflow.keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [ ]:
import gensim
import json
import pickle

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import plot_model
import keras.utils
from keras import utils as np_utils

#Keras Tokenizer just replaces certain punctuation characters and splits on the remaining space character.
#NLTK Tokenizer uses the Treebank tokenizer uses regular expressions to tokenize text as in Penn Treebank. 

In [ ]:
#WINDOWS_SIZE = 10
labels=['none','mild','moderate','moderately severe', 'severe']
num_classes = len(labels)

In [ ]:
#converting the transcript files into a dataframe
def transcripts_to_dataframe(directory):
    rows_list = []
        
    filenames = os.listdir(directory)
    
    if ".DS_Store" in filenames:
        filenames.remove(".DS_Store")
        
    for filename in filenames:
        transcript_path = os.path.join(directory, filename)
        transcript = pd.read_csv(transcript_path, sep='\t')
        m = re.search("(\d{3})_TRANSCRIPT.csv", filename)
        if m:
            person_id = m.group(1)
            p = {}
            question = ""
            answer = ""
            lines = len(transcript)
            for i in range(0, lines):
                row = transcript.iloc[i]
                if (row["speaker"] == "Ellie") or (i == lines - 1):
                    p["personId"] = person_id
                    if "(" in str(question):
                        question = question[question.index("(") + 1:question.index(")")]
                    p["question"] = question
                    p["answer"] = answer
                    if question != "":
                        rows_list.append(p)
                    p = {}
                    answer = ""
                    question = row["value"]
                else:
                    answer = str(answer) + " " + str(row["value"])

    all_participants = pd.DataFrame(rows_list, columns=['personId', 'question', 'answer'])
    all_participants.to_csv(directory + 'all.csv', sep=',')
    print("File was created")
    return all_participants

In [ ]:
#loading the data
data_path = "/content/drive/My Drive/transcripts/"
all_participants = transcripts_to_dataframe(data_path)

File was created


In [ ]:
all_participants.head(20)

,personId,question,answer
0,301,hi i'm ellie thanks for coming in today i was ...,thank you
1,301,think of me as a friend i don't judge i can't ...,mmm k
2,301,i'm here to learn about people and would love ...,
3,301,how are you doing today,i'm doing good thank you
4,301,that's good,
5,301,where are you from originally,i'm from los angeles
6,301,really me too,oh great
7,301,where do you live,i live in west los angeles the west side
8,301,nice,
9,301,how do you like your living situation,it's alright i xxx


In [ ]:
#Selecting the data from the data from based on the questions asked  ['where', 'when', 'how', 'why', 'are', 'what', 'do', 'have', 'can', 'did', 'is', 'could', 'so', 'tell', 'who', 'has']
interrogative = ["where", "when", "how","why","are","what","do","have","can","did","is", "could", "so", "tell", "who", "has"]
rslt_df = all_participants[all_participants.question.str.contains('|'.join(interrogative),na=False)]
rslt_df['answer'].replace('', np.nan, inplace=True)
rslt_df.dropna(subset = ["answer"], inplace=True)
rslt_df.reset_index(drop=True, inplace=True)
rslt_df
#rslt_df = rslt_df.to_csv("/content/drive/My Drive/rslt_df.csv")

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6746: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,personId,question,answer
0,301,think of me as a friend i don't judge i can't ...,mmm k
1,301,how are you doing today,i'm doing good thank you
2,301,where are you from originally,i'm from los angeles
3,301,where do you live,i live in west los angeles the west side
4,301,how do you like your living situation,it's alright i xxx
...,...,...,...
452,307,what's one of your most memorable experiences,[laughter] that's a tough question um probabl...
453,307,awesome,yeah
454,307,how would your best friend describe you,bubbly fun maybe moody you never know what ...
455,307,when was the last time you felt really happy,huh really happy oh when i was in mexico ove...


In [ ]:
rslt_df.sample(10)

,personId,question,answer
85,300,how do you like l_a,i love it
335,309,what do you do now,um currently <curr> uh currently a student
229,304,how do you know them,um he's my boyfriend i've been knowing for wh...
222,304,how do you know them,um through high school
223,304,how close are you to them,we're pretty close we try to talk as often as...
434,307,what's your dream job,dang it that is a tough one how 'bout just ge...
395,308,do you feel that way often,yes it eh yes that happens often
108,300,is there anything you regret,i would say going to college right after high...
72,302,what do you enjoy about traveling,meeting new people different locations scener...
225,304,when was the last time that happened,when what happened i was just


In [ ]:
rslt_df['question']= rslt_df.apply(lambda row: "{0}?".format(row.question), axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
rslt_df

,personId,question,answer
0,301,think of me as a friend i don't judge i can't ...,mmm k
1,301,how are you doing today?,i'm doing good thank you
2,301,where are you from originally?,i'm from los angeles
3,301,where do you live?,i live in west los angeles the west side
4,301,how do you like your living situation?,it's alright i xxx
...,...,...,...
452,307,what's one of your most memorable experiences ?,[laughter] that's a tough question um probabl...
453,307,awesome?,yeah
454,307,how would your best friend describe you?,bubbly fun maybe moody you never know what ...
455,307,when was the last time you felt really happy?,huh really happy oh when i was in mexico ove...


In [ ]:
rslt_df['statements'] = rslt_df[['question', 'answer']].agg(':'.join, axis=1)
rslt_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,personId,question,answer,statements
0,301,think of me as a friend i don't judge i can't ...,mmm k,think of me as a friend i don't judge i can't ...
1,301,how are you doing today?,i'm doing good thank you,how are you doing today?: i'm doing good thank...
2,301,where are you from originally?,i'm from los angeles,where are you from originally?: i'm from los a...
3,301,where do you live?,i live in west los angeles the west side,where do you live?: i live in west los angeles...
4,301,how do you like your living situation?,it's alright i xxx,how do you like your living situation?: it's a...
...,...,...,...,...
452,307,what's one of your most memorable experiences ?,[laughter] that's a tough question um probabl...,what's one of your most memorable experiences ...
453,307,awesome?,yeah,awesome?: yeah
454,307,how would your best friend describe you?,bubbly fun maybe moody you never know what ...,how would your best friend describe you?: bubb...
455,307,when was the last time you felt really happy?,huh really happy oh when i was in mexico ove...,when was the last time you felt really happy?:...


In [ ]:
def load_avec_dataset_file(path,score_column):
    ds = pd.read_csv(path, sep=',')
    ds['level'] = pd.cut(ds[score_column], bins=[-1,0,5,10,15,25], labels=[0,1,2,3,4])  #cut function used to segregate array into bins 5 levels - 'none','mild','moderate','moderately severe', 'severe'
    ds['PHQ8_Score'] = ds[score_column]
    ds['cat_level'] = keras.utils.to_categorical(ds['level'], num_classes).tolist() #categorical levels 
    ds = ds[['Participant_ID', 'level', 'cat_level', 'PHQ8_Score']]
    ds = ds.astype({"Participant_ID": int, "level": int, 'PHQ8_Score': int})
    return ds

In [ ]:
train = load_avec_dataset_file('/content/drive/My Drive/Depression_detect/train_split_Depression_AVEC2017 (1).csv','PHQ8_Score')
dev = load_avec_dataset_file('/content/drive/My Drive/Depression_detect/dev_split_Depression_AVEC2017.csv','PHQ8_Score')
test = load_avec_dataset_file('/content/drive/My Drive/Depression_detect/full_test_split.csv','PHQ8_Score')
print("Size: train= {}, dev= {}, test= {}".format(len(train), len(dev), len(test)))
train.head()

Size: train= 107, dev= 35, test= 47


,Participant_ID,level,cat_level,PHQ8_Score
0,303,0,"[1.0, 0.0, 0.0, 0.0, 0.0]",0
1,304,2,"[0.0, 0.0, 1.0, 0.0, 0.0]",6
2,305,2,"[0.0, 0.0, 1.0, 0.0, 0.0]",7
3,310,1,"[0.0, 1.0, 0.0, 0.0, 0.0]",4
4,312,1,"[0.0, 1.0, 0.0, 0.0, 0.0]",2


In [ ]:
ds_total = pd.concat([train,dev,test],sort=True)
total_phq8 = len(ds_total)
print("Total size = {}".format(total_phq8))

Total size = 189


In [ ]:
ds_total

,PHQ8_Score,Participant_ID,cat_level,level
0,0,303,"[1.0, 0.0, 0.0, 0.0, 0.0]",0
1,6,304,"[0.0, 0.0, 1.0, 0.0, 0.0]",2
2,7,305,"[0.0, 0.0, 1.0, 0.0, 0.0]",2
3,4,310,"[0.0, 1.0, 0.0, 0.0, 0.0]",1
4,2,312,"[0.0, 1.0, 0.0, 0.0, 0.0]",1
...,...,...,...,...
42,0,467,"[1.0, 0.0, 0.0, 0.0, 0.0]",0
43,3,469,"[0.0, 1.0, 0.0, 0.0, 0.0]",1
44,3,470,"[0.0, 1.0, 0.0, 0.0, 0.0]",1
45,1,480,"[0.0, 1.0, 0.0, 0.0, 0.0]",1


In [ ]:
len(ds_total)

189

In [ ]:
def split_by_phq_level(ds):
    none_ds = ds[ds['level']==0]
    mild_ds = ds[ds['level']==1]
    moderate_ds = ds[ds['level']==2]
    moderate_severe_ds = ds[ds['level']==3]
    severe_ds = ds[ds['level']==4]
    return (none_ds, mild_ds, moderate_ds, moderate_severe_ds, severe_ds)

In [ ]:
none_ds, mild_ds, moderate_ds, moderate_severe_ds, severe_ds = split_by_phq_level(ds_total)
print("Quantity per none_ds: {}, mild_ds: {}, moderate_ds {}, moderate_severe_ds: {}, severe_ds {}".format(len(none_ds), len(mild_ds), len(moderate_ds), len(moderate_severe_ds), len(severe_ds)))

Quantity per none_ds: 26, mild_ds: 70, moderate_ds 47, moderate_severe_ds: 24, severe_ds 22


In [ ]:
b_none_ds = ds_total[ds_total['level']==0]
b_mild_ds = ds_total[ds_total['level']==1].sample(26)
b_moderate_ds = ds_total[ds_total['level']==2].sample(26)
b_moderate_severe_ds = ds_total[ds_total['level']==3]
b_severe_ds = ds_total[ds_total['level']==4]

ds_total_b = pd.concat([b_none_ds, b_mild_ds, b_moderate_ds, b_moderate_severe_ds, b_severe_ds])
ds_total_b

,PHQ8_Score,Participant_ID,cat_level,level
0,0,303,"[1.0, 0.0, 0.0, 0.0, 0.0]",0
41,0,363,"[1.0, 0.0, 0.0, 0.0, 0.0]",0
42,0,364,"[1.0, 0.0, 0.0, 0.0, 0.0]",0
43,0,366,"[1.0, 0.0, 0.0, 0.0, 0.0]",0
45,0,369,"[1.0, 0.0, 0.0, 0.0, 0.0]",0
...,...,...,...,...
9,18,332,"[0.0, 0.0, 0.0, 0.0, 1.0]",4
13,18,354,"[0.0, 0.0, 0.0, 0.0, 1.0]",4
23,17,405,"[0.0, 0.0, 0.0, 0.0, 1.0]",4
37,17,453,"[0.0, 0.0, 0.0, 0.0, 1.0]",4


In [ ]:
len(ds_total_b)

124

In [ ]:
ds_lp = ds_total.merge(rslt_df,left_on=ds_total.Participant_ID.astype(int), right_on=rslt_df.personId.astype(int))
ds_lp_b = ds_total_b.merge(rslt_df,left_on=ds_total_b.Participant_ID.astype(int).astype(int), right_on=rslt_df.personId.astype(int))
len(ds_lp)

457

In [ ]:
len(ds_lp)

457

In [ ]:
def distribute_instances(ds, split_in = [70,14,16]):
    ds_shuffled = ds.sample(frac=1)
    none_ds, mild_ds, moderate_ds, moderate_severe_ds, severe_ds = split_by_phq_level(ds_shuffled)
    eq_ds = dict()
    prev_none = prev_mild = prev_moderate = prev_moderate_severe = prev_severe = 0
    split = split_in
    for p in split:
        last_none = min(len(none_ds), prev_none + round(len(none_ds) * p/100))
        last_mild = min(len(mild_ds), prev_mild + round(len(mild_ds) * p/100))
        last_moderate = min(len(moderate_ds), prev_moderate + round(len(moderate_ds) * p/100))
        last_moderate_severe = min(len(moderate_severe_ds), prev_moderate_severe + round(len(moderate_severe_ds) * p/100))
        last_severe = min(len(severe_ds), prev_severe + round(len(severe_ds) * p/100))  
        eq_ds['d'+str(p)] = pd.concat([none_ds[prev_none: last_none], mild_ds[prev_mild: last_mild], moderate_ds[prev_moderate: last_moderate], moderate_severe_ds[prev_moderate_severe: last_moderate_severe], severe_ds[prev_severe: last_severe]])
        prev_none = last_none
        prev_mild = last_mild
        prev_moderate = last_moderate
        prev_moderate_severe = last_moderate_severe
        prev_severe = last_severe  
    return (eq_ds['d70'], eq_ds['d14'], eq_ds['d16'])

In [ ]:
train_lp, dev_lp, test_lp = distribute_instances(ds_lp)
#train_lp_b, dev_lp_b, test_lp_b = distribute_instances(ds_lp_b)


In [ ]:
train_lp = train_lp.sample(frac=1).reset_index(drop=True)
test_lp=test_lp.sample(frac=1).reset_index(drop=True)
dev_lp=dev_lp.sample(frac=1).reset_index(drop=True)

In [ ]:
train_lp.head(10)

,key_0,PHQ8_Score,Participant_ID,cat_level,level,personId,question,answer,statements
0,300,2,300,"[0.0, 1.0, 0.0, 0.0, 0.0]",1,300,why?,um no specific reason i just don't travel a l...,why?: um no specific reason i just don't trave...
1,301,3,301,"[0.0, 1.0, 0.0, 0.0, 0.0]",1,301,when was the last time you felt really happy?,um when was the last time i really felt happy...,when was the last time you felt really happy?:...
2,300,2,300,"[0.0, 1.0, 0.0, 0.0, 0.0]",1,300,i'm sorry?,[laughter] it's alright,i'm sorry?: [laughter] it's alright
3,304,6,304,"[0.0, 0.0, 1.0, 0.0, 0.0]",2,304,could you have done anything to avoid it ?,yes <laughter>,could you have done anything to avoid it ?: ye...
4,304,6,304,"[0.0, 0.0, 1.0, 0.0, 0.0]",2,304,tell me more about that?,i guess i've been hearing that lately how i b...,tell me more about that?: i guess i've been he...
5,306,0,306,"[1.0, 0.0, 0.0, 0.0, 0.0]",0,306,where are you from originally?,uh colorado,where are you from originally?: uh colorado
6,306,0,306,"[1.0, 0.0, 0.0, 0.0, 0.0]",0,306,okay what's the best thing about being a parent?,um just seeing something that you're creating...,okay what's the best thing about being a paren...
7,307,4,307,"[0.0, 1.0, 0.0, 0.0, 0.0]",1,307,how do you know them?,one's from college and the other one we found...,how do you know them?: one's from college and ...
8,303,0,303,"[1.0, 0.0, 0.0, 0.0, 0.0]",0,303,how are you at controlling your temper ?,i'm pretty much good because see by me being ...,how are you at controlling your temper ?: i'm ...
9,300,2,300,"[0.0, 1.0, 0.0, 0.0, 0.0]",1,300,how did you feel in that moment?,uh frustrated,how did you feel in that moment?: uh frustrated


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub


In [ ]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [ ]:
!pip install sentencepiece
import tokenization

     |████████████████████████████████| 1.0MB 4.6MB/s 


In [ ]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
from keras.regularizers import l2
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    hidden1=Dense(64, kernel_regularizer=l2(0.03), bias_regularizer=l2(0.03) ,activation='relu')(clf_output)
    out = Dense(5 ,activation='softmax')(hidden1)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_cased_L-24_H-1024_A-16/2"
bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 18.6 s, sys: 3.92 s, total: 22.5 s
Wall time: 33.2 s


In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
train_input = bert_encode(train_lp.answer.values, tokenizer, max_len=160)
dev_input = bert_encode(dev_lp.answer.values, tokenizer, max_len=160)
test_input = bert_encode(test_lp.answer.values, tokenizer, max_len=160)

train_labels = train_lp.cat_level.values
test_labels=test_lp.cat_level.values
dev_labels=dev_lp.cat_level.values

In [ ]:
#train_labels=np.array(train_labels)
#dev_labels=np.array(dev_labels)
#test_labels=np.array(test_labels)

In [ ]:
print(train_labels[0])
print(type(train_input))
print(type(train_input[0]))
print(type(train_labels))
print(type(train_labels[0]))

[0.0, 1.0, 0.0, 0.0, 0.0]
<class 'tuple'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'list'>


In [ ]:

train_labels1=[]
for i in train_labels:
  train_labels1.append( np.array(i))

train_labels=np.array(train_labels1)

#print(train_labels)



dev_labels1=[]
for i in dev_labels:
  dev_labels1.append( np.array(i))

dev_labels=np.array(dev_labels1)

#print(dev_labels)



test_labels1=[]
for i in test_labels:
  test_labels1.append( np.array(i))

test_labels=np.array(test_labels1)

#print(train_labels)



In [ ]:
dev_input[0].size

10080

In [ ]:
test_input[0].size

11520

In [ ]:
model = build_model(bert_layer, max_len=160)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 333579265   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

3

In [ ]:
filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
train_input

(array([[  101, 15276,  1185, ...,     0,     0,     0],
        [  101, 15276,  1165, ...,     0,     0,     0],
        [  101,   164,  7053, ...,     0,     0,     0],
        ...,
        [  101,  1185,   102, ...,     0,     0,     0],
        [  101, 14863,  2608, ...,     0,     0,     0],
        [  101,  8147,   102, ...,     0,     0,     0]]),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]))

In [ ]:

train_history = model.fit(
    train_input, train_labels,
    validation_data=(dev_input,dev_labels),
    batch_size=16,
    epochs=10,
    callbacks=callbacks_list

)

model.save('model.h5')

Epoch 1/10
20/20 [==============================] - ETA: 0s - loss: 5.3370 - accuracy: 0.1944
Epoch 00001: val_accuracy improved from -inf to 0.23810, saving model to weights-improvement-01-0.24.hdf5
20/20 [==============================] - 95s 5s/step - loss: 5.3370 - accuracy: 0.1944 - val_loss: 5.2948 - val_accuracy: 0.2381
Epoch 2/10
20/20 [==============================] - ETA: 0s - loss: 5.1793 - accuracy: 0.3041
Epoch 00002: val_accuracy improved from 0.23810 to 0.30159, saving model to weights-improvement-02-0.30.hdf5
20/20 [==============================] - 94s 5s/step - loss: 5.1793 - accuracy: 0.3041 - val_loss: 5.2167 - val_accuracy: 0.3016
Epoch 3/10
20/20 [==============================] - ETA: 0s - loss: 5.0777 - accuracy: 0.3542
Epoch 00003: val_accuracy improved from 0.30159 to 0.34921, saving model to weights-improvement-03-0.35.hdf5
20/20 [==============================] - 103s 5s/step - loss: 5.0777 - accuracy: 0.3542 - val_loss: 5.1829 - val_accuracy: 0.3492
Epoch 

In [ ]:
test_pred = model.predict(test_input)

In [ ]:
print(test_pred)

[[0.1841854  0.5474751  0.13118438 0.0835271  0.05362801]
 [0.16828129 0.5131467  0.14177005 0.11743931 0.05936272]
 [0.11258365 0.63645566 0.07817651 0.11157141 0.06121273]
 [0.20757212 0.48984453 0.16286723 0.07168245 0.06803367]
 [0.1183167  0.46095082 0.11481845 0.04219804 0.26371595]
 [0.09319683 0.40376857 0.22475472 0.09629522 0.18198471]
 [0.1412621  0.36729705 0.2530121  0.09817425 0.14025444]
 [0.16109219 0.3921619  0.21465878 0.0919468  0.14014027]
 [0.21660182 0.26918942 0.31962177 0.14736907 0.04721792]
 [0.19412087 0.35874987 0.24527386 0.11725454 0.08460087]
 [0.2213484  0.4210125  0.2065774  0.07026854 0.0807932 ]
 [0.18351904 0.2760552  0.25654805 0.14080471 0.14307302]
 [0.23391707 0.4997168  0.06421568 0.10570125 0.09644914]
 [0.2515562  0.46157137 0.07560249 0.0448434  0.16642652]
 [0.06503819 0.3301228  0.42821944 0.13811873 0.03850085]
 [0.20623566 0.4072631  0.20051245 0.13033529 0.05565356]
 [0.10055017 0.17977901 0.63769025 0.06486443 0.0171161 ]
 [0.27384928 0

In [ ]:
score = model.evaluate(test_input, test_labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:',score[1] )

Test loss: 5.093750953674316
Test accuracy: 0.3611111044883728
